# TP 01: MR image handling, co-registration, intensity normalization

The aim of this session is to get familiar with MR images, and to perform some basic pre-processing step.
Python use is required. Exercises are indicated in red. 

In [1]:
#System imports
from __future__ import print_function
from __future__ import division
import os
import glob
import json
import collections
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
#Extra import
%matplotlib inline
import numpy as np
import SimpleITK as sitk
from joblib import Parallel, delayed
import nibabel as nib
from matplotlib import pyplot as plt
from scipy import ndimage

## Get familiar with MR images

The images we propose to load are in nifti format. This format embeds two kinds of information: the image itself (store in the data array), and some complementary about its acquisition (store in the header).
The images can be seen under three views: axial, coronal or sagittal.
Note also that the images you will see in this session and the next one show lesions.

In [3]:
datapath = os.path.join(os.path.dirname(os.getcwd()), "training")

In [4]:
impath = os.path.join(datapath, "1", "orig", "reg_T1.nii.gz")
print(impath)

C:\Users\helen\Documents\enseignement\epita2019\IMED\training\1\orig\reg_T1.nii.gz


In [5]:
#Load the MR volume
im = nib.load(impath)
im_arr = im.get_data()
#print(np.unique(im_arr))
print("Image dimension:", im.shape)

#Get the voxel size
print("Voxel size (mm):", im.header.get_zooms())
sx, sy, sz = im.shape

def show_axial(im_arr, mr_slice):
    plt.imshow(im_arr[:, :, mr_slice].T, cmap="gray")
    plt.axis('off')
    plt.pause(0.1)

#Show the MR volume, slice by slice, in axial view
interactive_plot = interactive(show_axial, im_arr=fixed(im_arr), mr_slice=(0, sz, 1))
interactive_plot    

Image dimension: (240, 240, 48)
Voxel size (mm): (0.958333, 0.958333, 3.0)


interactive(children=(IntSlider(value=24, description='mr_slice', max=48), Output()), _dom_classes=('widget-in…

MR imaging offers multiple advantages:
- no X-rays,
- good contrast in soft tissues,
- availability of different image types in one machine (multimodality)...

For example, you are vizualizing a T1 weighted MR image, slice by slice. The darkest parts of the images represent water and the brightest fat. Here, contrast agent has been previously injected to the patient before his scan, more specifically Gadolinium, allowing to enhance the image, which will be particularly useful to better delineate tumors, as we will see later.
Moreover, MR imaging is useful to make the distinction between the different parts of the brain. 
Let's now explore the brain !

In [6]:
#Load the manual segmentation of multiple tissues for the current patient
gtpath = os.path.join(datapath, "1", "segm.nii.gz")
gt = nib.load(gtpath)
gt_arr = gt.get_data().astype(int)

#Create the tissue list
tissue_types = ["Background", "Cortical gray matter", "Basal ganglia", "White matter", "White matter lesions", "Cerebrospinal fluid", 
                "Ventricles", "Cerebellum", "Brain stem", "Infarction", "Other"]

#Define the function to superimpose the MR image and regions of interest contours in axial view
def show_manual_seg(im_arr, seg_arr, mr_slice, tissue_type, tissue_types):
    tissue_label = np.where(np.array(tissue_types) == tissue_type)[0][0]
    tissue_arr = (seg_arr[:, :, mr_slice] == tissue_label).astype(int)
    plt.imshow(im_arr[:, :, mr_slice].T, cmap="gray")
    if np.count_nonzero(tissue_arr) > 0:
        plt.contour(tissue_arr.T, colors="r", linewidths=1, levels=[0.5, 1])
    plt.axis('off')
    plt.suptitle(tissue_types[tissue_label], fontsize=15)
    plt.pause(0.1)
    
    
#Show the MR volume, slice by slice, in axial view
interactive_plot = interactive(show_manual_seg, im_arr=fixed(im_arr), seg_arr=fixed(gt_arr), 
                               mr_slice=(0, sz - 1, 1), tissue_type=tissue_types, tissue_types=fixed(tissue_types))
interactive_plot    

interactive(children=(IntSlider(value=24, description='mr_slice', max=48), Dropdown(description='tissue_type',…

The axial view is particularly interesting for praticians, as its allows vizualizing the relative symmetry of the brain. Moreover, a brain lesion presence may be highlighted by a region of abnormal signal compared to the symmetrical region with respect to the inter-hemispheric plane (named the contralateral).  
Depending on the shape of the structure of interest to study, the other views may also be useful to show.

<font color='red'>Show the image in sagittal view</font> 

In [8]:
def show_sagittal(im_arr, mr_slice):
    plt.imshow(np.flipud(im_arr[mr_slice, :, :].T), cmap="gray")
    plt.axis('off')
    plt.pause(0.1)

interactive_plot = interactive(show_sagittal, im_arr=fixed(im_arr), mr_slice=(0, sx - 1, 1))
interactive_plot   

interactive(children=(IntSlider(value=120, description='mr_slice', max=240), Output()), _dom_classes=('widget-…

<font color='red'>Show the image in coronal view.</font> 

In [10]:
def show_coronal(im_arr, mr_slice):
    plt.imshow(np.flipud(im_arr[:, mr_slice, :].T), cmap="gray")
    plt.axis('off')
    plt.pause(0.1)

interactive_plot = interactive(show_coronal, im_arr=fixed(im_arr), mr_slice=(0, sy - 1, 1))
interactive_plot   

interactive(children=(IntSlider(value=120, description='mr_slice', max=240), Output()), _dom_classes=('widget-…

Voxel size is important when dealing with MR images. We are now going to prove it with two examples...

<font color='red'>Compute the ventricules volume, before and after downsampling (factor 2) the manual segmentation image. Comment. What should you do to make these volumes similar ?</font> 

<font color='red'>Compute the distance between the two basal ganglia, before and after downsampling (factor 2) the manual segmentation image. Comment. What should you do to make these distances similar ?</font> 

# Intensity normalization

Normalize the intensity is a common preprocessing step in MR imaging as there is not standard scale. It is also a mandatory step to further extract texture parameters, or in supervized segmentation. Try some methods below.

In [77]:
#Linear image quantization with grey levels in [0, 1]
norm_im_arr = im_arr / np.max(im_arr)

#Affine image quantization on 255 grey levels
norm_im_arr = 254 * (im_arr - np.min(im_arr)) / (np.min(im_arr) - np.max(im_arr))

#Zero-mean and unit variance
mu = np.mean(im_arr)
std = np.std(im_arr)
norm_im_arr = (im_arr - mu) / std

Normalize the MR image to have zero-mean and unit variance is commonly use in Deep learning segmentation methods (see for example the Deepmedic paper from Kamnitsask et. al, 2016). The code above normalizes the intensities on the whole image, but you can restrict the normalization to a specific tissue mask.

<font color='red'>Modify the code above to have zero-mean and unit variance within the brain mask, computed by excluding the background and the cerebrospinal fluid from the manual segmentation.</font> 

<font color='red'> Normalize all the T1 images to have zero-mean and unit variance within the brain mask. Superimpose the obtained histograms. Are they strictly the same ? Compute a histogram matching algorithm so that the histogram of image 2 is the same as the one of image 1.</font> 

In [78]:
def histogram_matching(im_arr, ref_arr, mask_arr):

    #Get the intensities of the reference and the target images
    #inside the reference mask
    [xM, yM, zM] = np.where(mask_arr != 0)
    im_gls = []
    ref_gls = []
    for ind_i, xi in enumerate(xM):
        yi = yM[ind_i]
        zi = zM[ind_i]
        im_gls.append(im_arr[xi, yi, zi])
        ref_gls.append(ref_arr[xi, yi, zi])

    #Sort the intensities of the reference image
    ref_sorted = sorted(ref_gls)
    im_sorted_ind = np.argsort(im_gls)

    #Process the target image intensities using the sorted reference
    #image intensities
    im_match_arr = np.zeros(im_arr.shape)
    for pos, ind_s in enumerate(im_sorted_ind):
        xs = xM[ind_s]
        ys = yM[ind_s]
        zs = zM[ind_s]
        im_match_arr[xs, ys, zs] = ref_sorted[pos]

    return im_match_arr

These methods are general, but some are specific to MR imaging (see Shinohara et al., Neuroimage, 2014)

## Co-registration

### Inter-modality and intra-patient co-registration

Images are acquired from the same modality, but from different patients.

<font color='red'>Complet the following code to perform an ICP algorithm to co-register the brain surfaces on T1 images. Define a score to quantitavely assess the quality of the registration.</font> 

In [104]:
#We are going to perform a simple co-registration algorithm, supposing that that two images only differ from a translation.
#The translation transformation has 3 parameters, so only one voxel matching is needed to find 
#the best transformation parameter.

impath1 = os.path.join(datapath, "1", "orig", "FLAIR.nii.gz")
im1 = nib.load(impath1)
im1_arr = im1.get_data()
print(im1.shape, im1.header.get_zooms())

impath2 = os.path.join(datapath, "5", "orig", "FLAIR.nii.gz")
im2 = nib.load(impath2)
im2_arr = im2.get_data()
print(im2.shape, im2.header.get_zooms())

#Load the manual segmentations
gtpath1 = os.path.join(datapath, "1", "segm.nii.gz")
gt1 = nib.load(gtpath1)
gt1_arr = gt1.get_data().astype(int)

gtpath2 = os.path.join(datapath, "4", "segm.nii.gz")
gt2 = nib.load(gtpath2)
gt2_arr = gt2.get_data().astype(int)

#Compute each brain mask
brain1_arr = (gt1_arr > 0).astype(int)
brain2_arr = (gt2_arr > 0).astype(int)

#Compute each brain surface array
d1 = ndimage.distance_transform_edt(brain1_arr).astype(int)
bs1_arr = np.logical_and(d1 <= 1, d1 > 0).astype(int)
d2 = ndimage.distance_transform_edt(brain2_arr).astype(int)
bs2_arr = np.logical_and(d2 <= 1, d2 > 0).astype(int)

#Get the center of mass of the brain surface of image 2
[x2, y2, z2] = np.where(bs2_arr > 0)
x2_0 = int(x2.sum() / len(x2))
y2_0 = int(y2.sum() / len(y2))
z2_0 = int(z2.sum() / len(z2))

previous_error = 0.5
error = 10000
reg_arr = np.copy(bs1_arr)

while error != previous_error:
    
    previous_error = error
    
    #Get the center of mass of each brain surface
    [x1, y1, z1] = np.where(reg_arr > 0)
    x1_0 = int(x1.sum() / len(x1))
    y1_0 = int(y1.sum() / len(y1))
    z1_0 = int(z1.sum() / len(z1))

    #Get the parameters of the translation from M1 to M2
    tx = x2_0 - x1_0
    ty = y2_0 - y1_0
    tz = z2_0 - z1_0

    #Compute the translated image 1 
    reg_arr = np.zeros((sx, sy, sz))
    for x, y, z in zip(x1, y1, z1):
        #print(tx, ty, tz)
        reg_arr[x + tx, y + ty, z + tz] = 1

    #Compute the mean square error between the translated image 1 and image 2
    error = ((reg_arr - bs2_arr) ** 2).sum()
    print(error)
    
#Quantitative assessment

(240, 240, 48) (0.958333, 0.958333, 3.0)
(240, 240, 48) (0.958333, 0.958333, 3.0)
99711.0
99711.0
0 0 0


### Intra-modality and inter-patient co-registration

Images acquired on the same machine, during the same acquisition session, but from different modalities.

A common strategy in co-registration consists in several step:
- criterion definition,
- transformation parameter initialization,
- transformation parameters optimization to optimize the criterion between the fixed and co-registered images.

To simplify the calculus, we suppose that the searched transformation is a translation.

<font color='red'>Complet the following code to co-register T1 and FLAIR images of the same patient optimizating a mutual information criterion.</font> 

In [ ]:
#Define a score to compare the co-registered image to the fixed image (here mutual information)
def mutual_info(im_arr, coreg_arr):
    
    return score

def translate(moving_arr, tx, ty, tz):
    
    return coreg_arr

#Optimize

## Extension

The co-registration we implemented included only a translation transform to simplify the calculation. However, it is not that simple when dealing with medical imaging. More approprieted transformations includes:
- rigid (rotation + translation),
- affine (rotation + translation + scaling),
- non linear co-registration ...

Many Python librairies offer robust medical imaging pre-processing, including co-registration algorithms:
- FSL,
- FreeSurfer,
- ANTs ...
We propose here to try the ones from ITK, but you are welcome to test other librairies.

<font color='red'> Test methods from Python librairies, intra-modality/inter-patient or inter-modality/inter-patient, rigid, affine or non-linear. Do you improve the co-registration performances ? You may also use the T1 standard image.</font> 

### Intra-modality and inter-patient co-registration

In [105]:
#Load the FLAIR image
flairpath = os.path.join(datapath, "1", "orig", "FLAIR.nii.gz")
flair = nib.load(flairpath)
flair_arr = flair.get_data()

#Load the native T1Gd image
t1path = os.path.join(datapath, "1", "orig", "T1.nii.gz")
t1 = nib.load(t1path)
t1_arr = t1.get_data()

fixed_image=sitk.ReadImage(flairpath,sitk.sitkFloat32)
moving_image=sitk.ReadImage(t1path,sitk.sitkFloat32)

initial_transform=sitk.CenteredTransformInitializer(fixed_image, moving_image, sitk.Euler3DTransform(),
                                                    sitk.CenteredTransformInitializerFilter.GEOMETRY)

registration_method=sitk.ImageRegistrationMethod()
# Similarity metric settings.
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkLinear)
# Optimizer settings.
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0,numberOfIterations=100,
                                                  convergenceMinimumValue=1e-6,convergenceWindowSize=10)
#registration_method.
#SetOptimizerScalesFromPhysicalShift()

final_transform=registration_method.Execute(fixed_image,moving_image)

moving_resampled=sitk.Resample(moving_image,fixed_image,final_transform,
                               sitk.sitkLinear,0.0,moving_image.GetPixelID())

print(type(moving_resampled))
move_arr = sitk.GetArrayFromImage(moving_resampled)
print(move_arr.shape)
print(flair_arr.shape)

reg_t1_arr = np.zeros((flair_arr.shape))
for mr_slice in range(0, sz):
    reg_t1_arr[:, :, mr_slice] = move_arr[mr_slice, :, :]

def show_axial2(im_arr, mr_slice):
    plt.imshow(im_arr[:, :, mr_slice], cmap="gray")
    plt.axis('off')
    plt.pause(0.1)
    
print(flair_arr.shape)
print(reg_t1_arr.shape)
#Show the MR volume, slice by slice, in axial view
interactive_plot = interactive(show_axial2, im_arr=fixed(reg_t1_arr), mr_slice=(0, sz, 1))
interactive_plot 

<class 'SimpleITK.SimpleITK.Image'>
(48, 240, 240)
(240, 240, 48)
(240, 240, 48)
(240, 240, 48)


interactive(children=(IntSlider(value=24, description='mr_slice', max=48), Output()), _dom_classes=('widget-in…

### Inhomogeneity correction

You may try the N4BiasFieldCorrection function to perform bias correction on MR images. Compare the obtained images. 